Some comments

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os
import numpy as np
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
assert os.path.isdir(repo_dir)

import lsst.daf.persistence as daf_persistence
butler = daf_persistence.Butler(repo_dir)

In [5]:
coadd_skymap = butler.get('deepCoadd_skyMap')

In [6]:
dir(coadd_skymap)

['ConfigClass',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_decToRingNum',
 '_numTracts',
 '_raStart',
 '_raToTractNum',
 '_ringNums',
 '_ringSize',
 '_sha1',
 '_tractCache',
 '_tractInfo',
 '_tractInfoList',
 '_version',
 '_wcsFactory',
 'config',
 'findAllTracts',
 'findClosestTractPatchList',
 'findTract',
 'findTractPatchList',
 'generateTract',
 'getRingIndices',
 'getSha1',
 'logSkyMapInfo',
 'register',
 'updateSha1']

In [42]:
help(coadd_skymap.findTractPatchList)

Help on method findTractPatchList in module lsst.skymap.ringsSkyMap:

findTractPatchList(coordList) method of lsst.skymap.ringsSkyMap.RingsSkyMap instance
    Find tracts and patches that overlap a region.
    
    Parameters
    ----------
    coordList : `list` of `lsst.geom.SpherePoint`
        List of ICRS sky coordinates to search for.
    
    Returns
    -------
    reList : `list` of (`TractInfo`, `list` of `PatchInfo`)
        For tracts and patches that contain, or may contain, the specified
        region. The list will be empty if there is no overlap.
    
    Notes
    -----
    **warning:**
        This uses a naive algorithm that may find some tracts and patches
        that do not overlap the region (especially if the region is not a
        rectangle aligned along patch x, y).



In [10]:
import h5py
spatial_lookup_filename = '/project/danielsf/valid_hsc_visit_extent.h5'
assert os.path.isfile(spatial_lookup_filename)
spatial_lookup = h5py.File(spatial_lookup_filename, 'r')

In [33]:
spatial_lookup['visit'][11625]

6316

In [34]:
visit_num = 6316
all_visit = np.where(spatial_lookup['visit'][()]==visit_num)

In [37]:
ra_min = spatial_lookup['ra_min'][()][all_visit].min()
ra_max = spatial_lookup['ra_max'][()][all_visit].max()
dec_min = spatial_lookup['dec_min'][()][all_visit].min()
dec_max = spatial_lookup['dec_max'][()][all_visit].max()

In [40]:
all_coverage = np.where(np.logical_and(spatial_lookup['ra_center'][()]>ra_min,
                        np.logical_and(spatial_lookup['ra_center'][()]<ra_max,
                        np.logical_and(spatial_lookup['dec_center'][()]>dec_min,
                                       spatial_lookup['dec_center'][()]<dec_max))))

In [41]:
np.unique(spatial_lookup['filter'][()][all_coverage])

array([b'HSC-G', b'HSC-I', b'HSC-R', b'HSC-Y', b'HSC-Z'], dtype='|S5')

In [43]:
import lsst.geom as lsst_geom

In [50]:
pt_list = []
for ra in np.arange(ra_min, ra_max, 0.005):
    for dec in np.arange(dec_min, dec_max, 0.005):
        pt_list.append(lsst_geom.SpherePoint(ra, dec, lsst_geom.radians))

In [52]:
tract_patch_list = coadd_skymap.findTractPatchList(pt_list)

In [64]:
type(tract_patch_list[0][1][0])

lsst.skymap.patchInfo.PatchInfo

In [97]:
data_id = {'tract':tract_patch_list[0][0].getId(),
           'patch': str(tract_patch_list[0][1][0].getIndex()).replace('(','').replace(')','').replace(' ',''),
           'filter':'HSC-Y'}

In [70]:
butler.getKeys(datasetType='deepCoadd')

{'filter': str, 'tract': int, 'patch': str}

In [116]:
import copy
data_id_g = copy.deepcopy(data_id)
data_id_g['filter'] = 'HSC-G'
data_id_r = copy.deepcopy(data_id)
data_id_r['filter'] = 'HSC-R'
data_id_i = copy.deepcopy(data_id)
data_id_i['filter'] = 'HSC-I'


In [115]:
data_id_g

{'tract': 9451, 'patch': '0,5', 'filter': ['HSC-G']}

In [117]:
data_g = butler.get('deepCoadd_forced_src', dataId=data_id_g).asAstropy()
data_r = butler.get('deepCoadd_forced_src', dataId=data_id_r).asAstropy()
data_i = butler.get('deepCoadd_forced_src', dataId=data_id_i).asAstropy()

In [106]:
for name in data.colnames:
    if 'endedness' in name:
        print(name)

base_ClassificationExtendedness_value
base_ClassificationExtendedness_flag


In [100]:
len(data)

24392

In [139]:
stars = np.where(data_g['base_ClassificationExtendedness_value']==0)

In [120]:
stars_r = np.where(data_r['base_ClassificationExtendedness_value']==0)
data_g['id'][:10]

41565959051280385
41565959051280386
41565959051280387
41565959051280389
41565959051280390
41565959051280391
41565959051280392
41565959051280393
41565959051280394
41565959051280395


In [136]:
np.testing.assert_array_equal(data_g['id'], data_r['id'])
np.testing.assert_array_equal(data_g['id'], data_i['id'])

In [124]:
stars_r[0]

array([   19,    37,    47, ..., 24382, 24386, 24387])

In [125]:
stars[0]

array([   37,    49,    52, ..., 24382, 24385, 24389])

In [126]:
len(stars_r[0])

5309

In [127]:
len(stars[0])

4563

In [129]:
stars_r[0][3]

49

In [134]:
np.any(data_g['base_ClassificationExtendedness_flag'][stars])

False

In [152]:
stars = np.where(np.logical_and(data_g['base_ClassificationExtendedness_value']==0,
                 np.logical_and(data_i['base_ClassificationExtendedness_value']==0,
                                data_r['base_ClassificationExtendedness_value']==0)))

In [153]:
len(stars[0])

2346

In [156]:
calib = butler.get('deepCoadd_calexp_photoCalib', dataId=data_id_r)

In [158]:
dir(calib)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_getFlux',
 '_getFluxMag0',
 '_getMagnitude',
 '_getThrowOnNegativeFlux',
 '_setThrowOnNegativeFlux',
 'calibrateCatalog',
 'calibrateImage',
 'computeScaledCalibration',
 'computeScalingTo',
 'getCalibrationErr',
 'getCalibrationMean',
 'getFlux',
 'getFluxMag0',
 'getInstFluxAtZeroMagnitude',
 'getMagnitude',
 'getThrowOnNegativeFlux',
 'instFluxToMagnitude',
 'instFluxToNanojansky',
 'isPersistable',
 'magnitudeToInstFlux',
 'readFits',
 'setFluxMag0',
 'setThrowOnNegativeFlux',
 'writeFits']

In [161]:
help(calib.instFluxToMagnitude)

Help on method instFluxToMagnitude in module lsst.afw.image.photoCalib:

instFluxToMagnitude(...) method of lsst.afw.image.photoCalib.PhotoCalib instance
    instFluxToMagnitude(*args, **kwargs)
    Overloaded function.
    
    1. instFluxToMagnitude(self: lsst.afw.image.photoCalib.PhotoCalib, instFlux: float, point: lsst.geom.Point2D) -> float
    
    2. instFluxToMagnitude(self: lsst.afw.image.photoCalib.PhotoCalib, instFlux: float) -> float
    
    3. instFluxToMagnitude(self: lsst.afw.image.photoCalib.PhotoCalib, instFlux: float, instFluxErr: float, point: lsst.geom.Point2D) -> lsst.afw.image.photoCalib.Measurement
    
    4. instFluxToMagnitude(self: lsst.afw.image.photoCalib.PhotoCalib, instFlux: float, instFluxErr: float) -> lsst.afw.image.photoCalib.Measurement
    
    5. instFluxToMagnitude(self: lsst.afw.image.photoCalib.PhotoCalib, sourceRecord: lsst.afw.table.SourceRecord, instFluxField: str) -> lsst.afw.image.photoCalib.Measurement
    
    6. instFluxToMagnitude(self